In [1]:
import random
import requests
import pandas as pd
from string import ascii_letters

In [2]:
df = pd.read_csv('../drug_datasets/dataset_files/drugs_dataset.csv')
medicines = list(df['medicine'].values)

In [3]:
def question_generator(drug):
    question = f"""\
When should you not take {drug}?
Is there any time when you shouldn't take {drug}?
Do you need to avoid taking {drug}?
When should you avoid taking {drug}?
What to avoid while taking {drug}?
When taking {drug}, what should I avoid?
What should one avoid when using {drug}?
Which things should you avoid while taking {drug}?
What are {drug} side-effects?
What are side-effects of {drug}
What condition is a serious side effect of {drug} use?
What are the dangers of {drug}?
{drug}
It is {drug}
I'm looking for {drug} detail
What is {drug}?
Can you give me information about {drug}?
Are you able to provide me with information about {drug}?
How should I take {drug}?
Give me {drug} dosage information
I need to know the dosage of {drug}
Tell me the dosage of {drug}
Tell me about {drug} dosage
What other drugs will affect {drug}?
What drugs should not be taken with {drug}?
Which drugs have a drug interaction with {drug}?
What other drugs will affect {drug}?"""
    return question

In [4]:
# list of all changed medicines 
changed_medicines = []

In [5]:
# randomly change letters
for medicine in medicines:
    for i in range(5):
        inds = [i for i, letter in enumerate(medicine) if not letter.isspace()]
        ind = random.sample(inds, 1)

        lst = list(medicine)
        lst[ind[0]] = random.choice(ascii_letters)
        changed_medicines.append("".join(lst))

In [6]:
# randomly remove letters
for medicine in medicines:
    for i in range(2):
        inds = [i for i, letter in enumerate(medicine) if not letter.isspace()]
        ind = random.sample(inds, 1)

        lst = list(medicine)
        lst[ind[0]] = ''
        changed_medicines.append("".join(lst))

In [7]:
# Also append correct words 
changed_medicines += medicines

In [8]:
# list of all successful entity extractors
entity_extractors = []

In [9]:
for medicine in changed_medicines:
    questions = question_generator(medicine)
    for question in questions.split('\n'): 
        post_fields = {"text" : question}
        r = requests.post("http://localhost:5005/model/parse", json=post_fields)
        dic = r.json()
        entities = dic['entities']

        for entity in entities:
            entity_extractors.append(entity['extractor'])

In [9]:
len(changed_medicines)

7968

In [10]:
len(medicines)

996

In [12]:
extractors = set(entity_extractors)
extractor_statistic = dict(zip(extractors, [0] * len(extractors)))

In [13]:
# statistic of successful entity extractors
for extractor in extractors:
    extractor_statistic[extractor] = entity_extractors.count(extractor)

In [14]:
extractor_statistic

{'RegexEntityExtractor': 35505, 'CRFEntityExtractor': 90, 'DIETClassifier': 596, 'SpacyEntityExtractor': 23164}